<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/rb_jomorgan_summarisation_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **FLAN-T5-Large** is tested for text summarisation using JPMorgan Financial transcripts.

The model is applied to two different parts of the transcripts:

**1)** individual financial analyst questions

**2)** responses to these questions.

**Different prompts** are applied on the two points above to allow the extraction of tailored outputs that serve different purposes:

**1)** *prompt = f"Extract and summarize the key questions asked by analysts in the following text: {text}"*

**2)** *prompt = f"Generate a detailed and comprehensive summary that captures all key points: {text}"*

**3)** *prompt = f"Rewrite the following text into a concise and original summary while maintaining its key ideas: {text}"*

**ROUGE scores**, which measure alignment with reference texts through precision, recall, and F-measure, are used, helping assess models performance.


In [ ]:
!pip install bertopic umap-learn hdbscan sentence-transformers > /dev/null 2>&1
!pip install transformers torch > /dev/null 2>&1
!pip install rouge_score > /dev/null 2>&1
!pip install evaluate > /dev/null 2>&1
!pip install --upgrade protobuf > /dev/null 2>&1
!pip install tensorboard > /dev/null 2>&1
!pip install tensorflow > /dev/null 2>&1

In [ ]:
import time
import torch
from google.colab import drive
import os
import sys
import pandas as pd
import re
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import evaluate
from rouge_score import rouge_scorer
from typing import List, Union, Optional
import logging
import tensorflow as tf
import numpy as np
import random

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
def reset_session():
    tf.keras.backend.clear_session()
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)

In [ ]:

# Load data (questions and answers for JPM and UBS)

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:

path1 = "/content/drive/MyDrive/BOE/bank_of_england/data/preprocessed_data/jp_morgan_qna.csv"

path2 = "/content/drive/MyDrive/BOE/bank_of_england/data/preprocessed_data/ubs_qa_df_preprocessed_ver2.csv"

JP_qna = pd.read_csv(path1)
UBS_qna = pd.read_csv(path2)

In [ ]:
JP_qna = JP_qna[JP_qna["Quarter"] != "1Q23"]
UBS_qna = UBS_qna[UBS_qna["Quarter"] != "1Q23"]

##**Q&A summarisation**

### **Analysing Jim Mitchell data Q2-2024 data**

In [ ]:
reset_session()

In [ ]:
filtered_df = JP_qna[(JP_qna["Analyst"] == 'Jim Mitchell')& (JP_qna["Quarter"] == "4Q24")]

**Summarising questions**

In [ ]:
analyst_q = filtered_df["Question"].tolist()  #### genertaing list for modeling

In [ ]:

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TextSummarizer:
    def __init__(self, model_name: str = "google/flan-t5-large", device: Optional[str] = None):
        """Initialize the summarizer with model and device."""
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
            logger.info(f"Successfully loaded {model_name}")
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def chunk_text(self,
                  text: Union[str, List[str]],
                  chunk_size: int = 400,
                  overlap: int = 50) -> List[str]:
        """Split text into overlapping chunks."""
        if chunk_size <= 0 or overlap < 0 or overlap >= chunk_size:
            raise ValueError("Invalid chunk_size or overlap parameters")

        try:
            if isinstance(text, list):
                text = " ".join(text)

            if not text.strip():
                return []

            words = text.split()
            chunks = []
            start = 0

            while start < len(words):
                end = min(start + chunk_size, len(words))
                chunk = " ".join(words[start:end])
                chunks.append(chunk)
                start += chunk_size - overlap

            logger.debug(f"Split text into {len(chunks)} chunks")
            return chunks

        except Exception as e:
            logger.error(f"Error in chunk_text: {str(e)}")
            raise

    def summarize_text_q(self,
                      text: str,
                      min_new_tokens: int = 50,
                      max_new_tokens: int = 250) -> str:
        """Summarize a single piece of text focusing on analyst questions."""
        if pd.isna(text) or not text.strip():
            logger.warning("Empty or NaN text provided")
            return ""

        try:
            # Specialized prompt for analyst questions
            prompt = f"Extract and summarize the key questions asked by analysts in the following text: {text}"

            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    min_new_tokens=min_new_tokens,
                    max_new_tokens=max_new_tokens,
                    num_beams=4,
                    length_penalty=2.0,
                    no_repeat_ngram_size=3,
                    early_stopping=True,
                    do_sample=False
                )

            summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return summary

        except Exception as e:
            logger.error(f"Error in summarize_text: {str(e)}")
            raise

    def summarize_long_text_q(self,
                          text: Union[str, List[str]],
                          chunk_size: int = 400,
                          overlap: int = 50) -> str:
        """Handle long text summarization focusing on analyst questions."""
        try:
            chunks = self.chunk_text(text, chunk_size, overlap)
            if not chunks:
                logger.warning("No valid chunks to summarize")
                return ""

            chunk_summaries = []
            for i, chunk in enumerate(chunks):
                logger.debug(f"Summarizing chunk {i+1}/{len(chunks)}")
                summary = self.summarize_text_q(chunk)
                if summary.strip():
                    chunk_summaries.append(summary)

            if not chunk_summaries:
                logger.warning("No valid summaries generated")
                return ""

            if len(chunk_summaries) == 1:
                return chunk_summaries[0]

            logger.debug("Generating final summary")
            final_summary = self.summarize_text_q(
                " ".join(chunk_summaries),
                min_new_tokens=150,
                max_new_tokens=300
            )

            return final_summary

        except Exception as e:
            logger.error(f"Error in summarize_long_text: {str(e)}")
            raise

# Running the model
try:

    summarizer_q = TextSummarizer()


    logger.info("Starting summarization of analyst questions")
    question_summary = summarizer_q.summarize_long_text_q(analyst_q)


    print("\nSummary of Analyst Questions:")
    print(question_summary)

except Exception as e:
    logger.error(f"Error during summarization: {str(e)}")
    print(f"An error occurred: {str(e)}")


Summary of Analyst Questions:
Key questions: What areas of the regulatory structure, if it were to change, would be most impactful for you? And is there any areas where you think capital requirements could actually go down? Or is this more of a story of requirements just simply stop going up?


**Adding ROUGE score for valuation**

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
analyst_q_str = " ".join(analyst_q)

# Calculate ROUGE scores
scores_q = scorer.score(analyst_q_str, question_summary)
for key in scores_q:
    print(f'{key}: {scores_q[key]}')

rouge1: Score(precision=0.9574468085106383, recall=0.3333333333333333, fmeasure=0.4945054945054945)
rouge2: Score(precision=0.9565217391304348, recall=0.3283582089552239, fmeasure=0.48888888888888893)
rougeL: Score(precision=0.9574468085106383, recall=0.3333333333333333, fmeasure=0.4945054945054945)


ROUGE results reveal good model's ability to capture both the relevance and completeness of the content. The higher recall values, combined with solid precision, have resulted in strong F1 scores across all ROUGE metrics, demonstrating that the model is  producing comprehensive and accurate summaries.

### **Analysing John McDonald data Q2-2024 data**

In [ ]:
filtered_df3 = JP_qna[(JP_qna["Analyst"] == 'John McDonald')& (JP_qna["Quarter"] == "4Q24")]


**Summarinsing answsers**

In [ ]:
analyst_text2 = filtered_df2["Response"].tolist()  #### genertaing list for modeling

In [ ]:
reset_session()

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TextSummarizer:
    def __init__(self, model_name: str = "google/flan-t5-large", device: Optional[str] = None):
        """Initialize the summarizer with model and device."""
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
            logger.info(f"Successfully loaded {model_name}")
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def chunk_text(self,
                  text: Union[str, List[str]],
                  chunk_size: int = 400,
                  overlap: int = 50) -> List[str]:
        """Split text into overlapping chunks."""
        if chunk_size <= 0 or overlap < 0 or overlap >= chunk_size:
            raise ValueError("Invalid chunk_size or overlap parameters")

        try:
            if isinstance(text, list):
                text = " ".join(text)

            if not text.strip():
                return []

            words = text.split()
            chunks = []
            start = 0

            while start < len(words):
                end = min(start + chunk_size, len(words))
                chunk = " ".join(words[start:end])
                chunks.append(chunk)
                start += chunk_size - overlap

            logger.debug(f"Split text into {len(chunks)} chunks")
            return chunks

        except Exception as e:
            logger.error(f"Error in chunk_text: {str(e)}")
            raise

    def summarize_text(self,
                      text: str,
                      min_new_tokens: int = 100,
                      max_new_tokens: int = 500) -> str:
        """Summarize a single piece of text focusing on analyst questions."""
        if pd.isna(text) or not text.strip():
            logger.warning("Empty or NaN text provided")
            return ""

        try:
            # Specialized prompt for answer to analyst
            prompt = f"Generate a detailed and comprehensive summary that captures all key points: {text}"

            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    min_new_tokens=min_new_tokens,
                    max_new_tokens=max_new_tokens,
                    num_beams=4,
                    length_penalty=2.0,
                    no_repeat_ngram_size=3,
                    early_stopping=True,
                    do_sample=False
                )

            summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return summary

        except Exception as e:
            logger.error(f"Error in summarize_text: {str(e)}")
            raise

    def summarize_long_text(self,
                          text: Union[str, List[str]],
                          chunk_size: int = 400,
                          overlap: int = 50) -> str:
        """Handle long text summarization focusing on analyst questions."""
        try:
            chunks = self.chunk_text(text, chunk_size, overlap)
            if not chunks:
                logger.warning("No valid chunks to summarize")
                return ""

            chunk_summaries = []
            for i, chunk in enumerate(chunks):
                logger.debug(f"Summarizing chunk {i+1}/{len(chunks)}")
                summary = self.summarize_text(chunk)
                if summary.strip():
                    chunk_summaries.append(summary)

            if not chunk_summaries:
                logger.warning("No valid summaries generated")
                return ""

            if len(chunk_summaries) == 1:
                return chunk_summaries[0]

            logger.debug("Generating final summary")
            final_summary = self.summarize_text(
                " ".join(chunk_summaries),
                min_new_tokens=150,
                max_new_tokens=500
            )

            return final_summary

        except Exception as e:
            logger.error(f"Error in summarize_long_text: {str(e)}")
            raise


In [ ]:
# Running the model
try:

    summarizer = TextSummarizer()


    logger.info("Starting summarization of answer")
    answer_summary2 = summarizer.summarize_long_text(analyst_text2)


    print("\nSummary of Answer:")
    print(answer_summary2)

except Exception as e:
    logger.error(f"Error during summarization: {str(e)}")
    print(f"An error occurred: {str(e)}")


Summary of Answer:
"We would like to not have the excess grow from here," CEO Jeff Bezos tells CNBC in an interview with CNBC's Jim Cramer. "We've grown a lot over the last few years, and that's a form of efficiency. But what I would say is that if you look at the head count trajectory of the company, we have grown very much and it's been a very strong growth story over the past few years. But, at the margin, that means that inside the tech teams, there's got a little bit of capacity that gets freed up to focus on features and new product development and so on, which is also in some sense a kind of efficiency." The obvious exceptions are the ongoing areas of high certainty investment and growth, so, obviously, branches and bankers, and also, critical non-negotiable areas of risk and control like cyber.


In [ ]:
analyst_text2_str = " ".join(analyst_text2)

# Calculate ROUGE scores
scores2 = scorer.score(analyst_text2_str, answer_summary2)
for key in scores2:
    print(f'{key}: {scores2[key]}')

rouge1: Score(precision=0.9078947368421053, recall=0.14465408805031446, fmeasure=0.24954792043399637)
rouge2: Score(precision=0.7682119205298014, recall=0.12172088142707241, fmeasure=0.2101449275362319)
rougeL: Score(precision=0.6973684210526315, recall=0.1111111111111111, fmeasure=0.19168173598553348)


These ROUGE scores suggest that the generated summary captures key phrases with high precision but has low recall, meaning it includes mostly relevant words but misses many important details from the original text. The F1-scores indicate an overall weak balance, showing that the summary is quite selective and may need improvements to cover more content.

##**Tuning parameters and changing prompt to summarise Answers text**

In [ ]:
reset_session()

**Changing prompt and updating chunks sizes to capture better context from the text.**

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class TextSummarizer:
    def __init__(self, model_name: str = "google/flan-t5-large", device: Optional[str] = None):
        """Initialize the summarizer with model and device."""
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
            logger.info(f"Successfully loaded {model_name}")
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def chunk_text(self,
                  text: Union[str, List[str]],
                  chunk_size: int = 400,
                  overlap: int = 50) -> List[str]:
        """Split text into overlapping chunks."""

        if chunk_size <= 0 or overlap < 0 or overlap >= chunk_size:
            raise ValueError("Invalid chunk_size or overlap parameters")

        try:

            if isinstance(text, list):
                text = " ".join(text)

            if not text.strip():
                return []

            words = text.split()
            chunks = []
            start = 0

            while start < len(words):
                end = min(start + chunk_size, len(words))
                chunk = " ".join(words[start:end])
                chunks.append(chunk)
                start += chunk_size - overlap

            logger.debug(f"Split text into {len(chunks)} chunks")
            return chunks

        except Exception as e:
            logger.error(f"Error in chunk_text: {str(e)}")
            raise

    def summarize_text(self,
                      text: str,
                      min_new_tokens: int = 100,
                      max_new_tokens: int = 400) -> str:
        """Summarize a single piece of text."""

        if pd.isna(text) or not text.strip():
            logger.warning("Empty or NaN text provided")
            return ""

        try:
            prompt = f"Rewrite the following text into a concise and original summary while maintaining its key ideas: {text}"


            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(self.device)


            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    min_new_tokens=min_new_tokens,
                    max_new_tokens=max_new_tokens,
                    num_beams=4,
                    length_penalty=2,
                    no_repeat_ngram_size=3,
                    early_stopping=True,
                    do_sample=False
                )

            summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return summary

        except Exception as e:
            logger.error(f"Error in summarize_text: {str(e)}")
            raise

    def summarize_long_text_2(self,
                          text: Union[str, List[str]],
                          chunk_size: int = 300,
                          overlap: int = 50) -> str:
        """Handle long text summarization."""
        try:
            # Get chunks
            chunks = self.chunk_text(text, chunk_size, overlap)
            if not chunks:
                logger.warning("No valid chunks to summarize")
                return ""

            # Summarize chunks
            chunk_summaries = []
            for i, chunk in enumerate(chunks):
                logger.debug(f"Summarizing chunk {i+1}/{len(chunks)}")
                summary = self.summarize_text(chunk)
                if summary.strip():
                    chunk_summaries.append(summary)

            if not chunk_summaries:
                logger.warning("No valid summaries generated")
                return ""


            if len(chunk_summaries) == 1:
                return chunk_summaries[0]

            # Summarize the combined summaries
            logger.debug("Generating final summary")
            final_summary = self.summarize_text(
                " ".join(chunk_summaries),
                min_new_tokens=150,
                max_new_tokens=300
            )

            return final_summary

        except Exception as e:
            logger.error(f"Error in summarize_long_text: {str(e)}")
            raise


In [ ]:
# Running the model
try:

    summarizer = TextSummarizer()


    logger.info("Starting summarization of answer")
    answer_summary3 = summarizer.summarize_long_text_2(analyst_text2)


    print("\nSummary of Answer:")
    print(answer_summary3)

except Exception as e:
    logger.error(f"Error during summarization: {str(e)}")
    print(f"An error occurred: {str(e)}")


Summary of Answer:
I think the way we're thinking about it right now is that we feel very comfortable with the notion that it makes sense for us to have a nice store of extra capital in light of the current environment. We believe there's a good chance that we get to deploy it at better levels essentially than the current opportunities would suggest. And so that feels like a correct kind of strategic and financial decision for us. Having said that, having studied it quite extensively over the last six months and have all the debates that you would expect, we've concluded that we do have enough. We have enough excess. And given that, we would like not have the excess grow from here. So, when you think about the implications of that, given the amount of organic capital generation that we’re producing, it means that unless we find in the near term opportunities for organic deployment or otherwise, it mean more capital return through buyback, all else being equal, in order to arrest the g

In [ ]:
analyst_text3_str = " ".join(analyst_text2)

# Calculate ROUGE scores
scores2 = scorer.score(analyst_text3_str, answer_summary3)
for key in scores2:
    print(f'{key}: {scores2[key]}')

rouge1: Score(precision=0.9948979591836735, recall=0.20440251572327045, fmeasure=0.33913043478260874)
rouge2: Score(precision=0.958974358974359, recall=0.19622245540398742, fmeasure=0.3257839721254356)
rougeL: Score(precision=0.5408163265306123, recall=0.1111111111111111, fmeasure=0.1843478260869565)


The overall results have now improved, although some of the key context is still missing, possibly due to chunking the data.

ROUGE-1 and ROUGE-2 show higher precision, recall, and F1-score, meaning the summary captures more relevant words and bigrams while maintaining good accuracy.

ROUGE-L recall remains the same, but ROUGE-L precision dropped, indicating some loss in capturing longer phrase structures.

Overall, the improved ROUGE-1 and ROUGE-2 scores suggest the summary is now more aligned with the original text while retaining conciseness.

**Adding longer chunks and re-running the model**

In [ ]:
reset_session()

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class TextSummarizer:
    def __init__(self, model_name: str = "google/flan-t5-large", device: Optional[str] = None):
        """Initialize the summarizer with model and device."""
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
            logger.info(f"Successfully loaded {model_name}")
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def chunk_text(self,
                  text: Union[str, List[str]],
                  chunk_size: int = 500,
                  overlap: int = 50) -> List[str]:
        """Split text into overlapping chunks."""

        if chunk_size <= 0 or overlap < 0 or overlap >= chunk_size:
            raise ValueError("Invalid chunk_size or overlap parameters")

        try:

            if isinstance(text, list):
                text = " ".join(text)

            if not text.strip():
                return []

            words = text.split()
            chunks = []
            start = 0

            while start < len(words):
                end = min(start + chunk_size, len(words))
                chunk = " ".join(words[start:end])
                chunks.append(chunk)
                start += chunk_size - overlap

            logger.debug(f"Split text into {len(chunks)} chunks")
            return chunks

        except Exception as e:
            logger.error(f"Error in chunk_text: {str(e)}")
            raise

    def summarize_text(self,
                      text: str,
                      min_new_tokens: int = 100,
                      max_new_tokens: int = 500) -> str:
        """Summarize a single piece of text."""

        if pd.isna(text) or not text.strip():
            logger.warning("Empty or NaN text provided")
            return ""

        try:
            prompt = f"Rewrite the following text into a concise and original summary while maintaining its key ideas: {text}"


            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(self.device)


            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    min_new_tokens=min_new_tokens,
                    max_new_tokens=max_new_tokens,
                    num_beams=4,
                    length_penalty=2,
                    no_repeat_ngram_size=3,
                    early_stopping=True,
                    do_sample=False
                )

            summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return summary

        except Exception as e:
            logger.error(f"Error in summarize_text: {str(e)}")
            raise

    def summarize_long_text_2(self,
                          text: Union[str, List[str]],
                          chunk_size: int = 500,
                          overlap: int = 50) -> str:
        """Handle long text summarization."""
        try:
            # Get chunks
            chunks = self.chunk_text(text, chunk_size, overlap)
            if not chunks:
                logger.warning("No valid chunks to summarize")
                return ""

            # Summarize chunks
            chunk_summaries = []
            for i, chunk in enumerate(chunks):
                logger.debug(f"Summarizing chunk {i+1}/{len(chunks)}")
                summary = self.summarize_text(chunk)
                if summary.strip():
                    chunk_summaries.append(summary)

            if not chunk_summaries:
                logger.warning("No valid summaries generated")
                return ""


            if len(chunk_summaries) == 1:
                return chunk_summaries[0]

            # Summarize the combined summaries
            logger.debug("Generating final summary")
            final_summary = self.summarize_text(
                " ".join(chunk_summaries),
                min_new_tokens=150,
                max_new_tokens=300
            )

            return final_summary

        except Exception as e:
            logger.error(f"Error in summarize_long_text: {str(e)}")
            raise

In [ ]:
# Running the model
try:

    summarizer = TextSummarizer()


    logger.info("Starting summarization of answer")
    answer_summary3 = summarizer.summarize_long_text_2(analyst_text2)


    print("\nSummary of Answer:")
    print(answer_summary3)

except Exception as e:
    logger.error(f"Error during summarization: {str(e)}")
    print(f"An error occurred: {str(e)}")


Summary of Answer:
Daniel: So, yeah, you've noted all the points that we always make so I won't repeat them. And I think the way we're thinking about it right now is that we feel very comfortable with the notion that it makes sense for us to have a nice store of extra capital in light of the current environment. We believe there's a good chance that there will be a moment where we get to deploy it at better levels than the current opportunities would suggest. And so that feels like a correct kind of strategic and financial decision for us. Having said that, having studied it quite extensively over the last six months and have all the debates that you would expect, we've concluded that we do have enough. We have enough excess. And given that, we would like not have the excess grow from here. And that is our current plan, although, I'll give the caveat that, as you know, is in our disclosure, which is we don't want to get in the business of guiding on buybacks.


In [ ]:
analyst_text3_str = " ".join(analyst_text2)

# Calculate ROUGE scores
scores2 = scorer.score(analyst_text3_str, answer_summary3)
for key in scores2:
    print(f'{key}: {scores2[key]}')

rouge1: Score(precision=1.0, recall=0.19392033542976939, fmeasure=0.32484635645302895)
rouge2: Score(precision=0.9782608695652174, recall=0.1888772298006296, fmeasure=0.31662269129287596)
rougeL: Score(precision=0.9945945945945946, recall=0.1928721174004193, fmeasure=0.3230904302019315)


Overall ROUGE results have improved after tuning chunks sizes.